
1. _Write data to delta lake (managed table)_
2. _Write data to delta lake (external table)_
3. _Read data from delta lake (Table)_
4. _Read data from delta lake (File)_

In [0]:
%sql
--DROP DATABASE f1_demo CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS f1_demo
LOCATION '/mnt/formula1dlsaga/demo'

In [0]:
results_df = spark.read.option("inferSchema", True).json("/mnt/formula1dlsaga/raw/2021-03-21/results.json")

In [0]:
results_df.write.format("delta").mode("overwrite").saveAsTable("f1_demo.results_managed")

In [0]:
%sql
--SELECT * FROM f1_demo.results_managed;
DESC TABLE EXTENDED f1_demo.results_managed;

col_name,data_type,comment
constructorId,bigint,null
driverId,bigint,null
fastestLap,string,null
fastestLapSpeed,string,null
fastestLapTime,string,null
grid,bigint,null
laps,bigint,null
milliseconds,string,null
number,string,null
points,double,null


In [0]:
results_df.write.format("delta").mode("overwrite").save("/mnt/formula1dlsaga/demo/results_external")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.results_external
USING DELTA
LOCATION '/mnt/formula1dlsaga/demo/results_external'

In [0]:
%sql
SELECT * FROM f1_demo.results_external;

In [0]:
results_external_df = spark.read.format("delta").load("/mnt/formula1dlsaga/demo/results_external")
display(results_external_df)

In [0]:
results_df.write.format("delta").mode("overwrite").partitionBy("constructorId").saveAsTable("f1_demo.results_partitioned")

In [0]:
%sql
SHOW PARTITIONS f1_demo.results_partitioned

constructorId
26
29
19
54
22
7
34
50
57
32



1. _Update Delta Table_
2. _Delete from Delta Table_

In [0]:
%sql 
UPDATE f1_demo.results_managed
SET points = 11 - position
WHERE position <= 10;

num_affected_rows
10081


In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, '/mnt/formula1dlsaga/demo/results_managed')

# Declare the predicate by using a SQL-formatted string.
deltaTable.update("position <= 10", {"points": "11 - position"})

In [0]:
%sql
DELETE FROM f1_demo.results_managed WHERE position > 10;

num_affected_rows
4144


In [0]:
%sql
SELECT * FROM f1_demo.results_managed;

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, '/mnt/formula1dlsaga/demo/results_managed')

# Declare the predicate by using a SQL-formatted string.
deltaTable.delete("points = 0")


## Upsert using merge

In [0]:
drivers_d1_df = spark.read.option("inferSchema", True).json("/mnt/formula1dlsaga/raw/2021-03-28/drivers.json").filter("driverId <= 10").select("driverId", "dob", "name.forename", "name.surname")

In [0]:
drivers_d1_df.createOrReplaceTempView("drivers_day1")

In [0]:
display(drivers_d1_df)


driverId,dob,forename,surname
1,1985-01-07,Lewis,Hamilton
2,1977-05-10,Nick,Heidfeld
3,1985-06-27,Nico,Rosberg
4,1981-07-29,Fernando,Alonso
5,1981-10-19,Heikki,Kovalainen
6,1985-01-11,Kazuki,Nakajima
7,1979-02-28,Sébastien,Bourdais
8,1979-10-17,Kimi,Räikkönen
9,1984-12-07,Robert,Kubica
10,1982-03-18,Timo,Glock


In [0]:
from pyspark.sql.functions import upper

drivers_d2_df = spark.read.option("inferSchema", True)\
                        .json("/mnt/formula1dlsaga/raw/2021-03-28/drivers.json")\
                        .filter("driverId BETWEEN 6 AND 15")\
                        .select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))


In [0]:
drivers_d2_df.createOrReplaceTempView("drivers_day2")

In [0]:
display(drivers_d2_df)

driverId,dob,forename,surname
6,1985-01-11,KAZUKI,NAKAJIMA
7,1979-02-28,SÉBASTIEN,BOURDAIS
8,1979-10-17,KIMI,RÄIKKÖNEN
9,1984-12-07,ROBERT,KUBICA
10,1982-03-18,TIMO,GLOCK
11,1977-01-28,TAKUMA,SATO
12,1985-07-25,NELSON,PIQUET JR.
13,1981-04-25,FELIPE,MASSA
14,1971-03-27,DAVID,COULTHARD
15,1974-07-13,JARNO,TRULLI


In [0]:
drivers_d3_df = spark.read.option("inferSchema", True)\
                        .json("/mnt/formula1dlsaga/raw/2021-03-28/drivers.json")\
                        .filter("driverId BETWEEN 1 AND 5 OR driverId BETWEEN 16 AND 20")\
                        .select("driverId", "dob", upper("name.forename").alias("forename"), upper("name.surname").alias("surname"))

In [0]:
display(drivers_d3_df)

driverId,dob,forename,surname
1,1985-01-07,LEWIS,HAMILTON
2,1977-05-10,NICK,HEIDFELD
3,1985-06-27,NICO,ROSBERG
4,1981-07-29,FERNANDO,ALONSO
5,1981-10-19,HEIKKI,KOVALAINEN
6,1985-01-11,KAZUKI,NAKAJIMA
7,1979-02-28,SÉBASTIEN,BOURDAIS
8,1979-10-17,KIMI,RÄIKKÖNEN
9,1984-12-07,ROBERT,KUBICA
10,1982-03-18,TIMO,GLOCK


In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_merge (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA


##Day 1

In [0]:
%sql
MERGE INTO f1_demo.drivers_merge AS tgt
USING drivers_day1 AS upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET
    tgt.dob = upd.dob,
    tgt.forename = upd.forename,
    tgt.surname = upd.surname,
    tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (
    driverId,
    dob,
    forename,
    surname,
    createdDate
  )
  VALUES (
    driverID,
    dob,
    forename,
    surname,
    current_timestamp
  )

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,0,0,10


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverId,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2025-11-21,null
2,1977-05-10,Nick,Heidfeld,2025-11-21,null
3,1985-06-27,Nico,Rosberg,2025-11-21,null
4,1981-07-29,Fernando,Alonso,2025-11-21,null
5,1981-10-19,Heikki,Kovalainen,2025-11-21,null
6,1985-01-11,Kazuki,Nakajima,2025-11-21,null
7,1979-02-28,Sébastien,Bourdais,2025-11-21,null
8,1979-10-17,Kimi,Räikkönen,2025-11-21,null
9,1984-12-07,Robert,Kubica,2025-11-21,null
10,1982-03-18,Timo,Glock,2025-11-21,null



## Day 2


In [0]:
%sql
MERGE INTO f1_demo.drivers_merge AS tgt
USING drivers_day2 AS upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
  UPDATE SET
    tgt.dob = upd.dob,
    tgt.forename = upd.forename,
    tgt.surname = upd.surname,
    tgt.updatedDate = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (
    driverId,
    dob,
    forename,
    surname,
    createdDate
  )
  VALUES (
    driverID,
    dob,
    forename,
    surname,
    current_timestamp
  )

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10,5,0,5


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverId,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,LEWIS,HAMILTON,2025-11-21,2025-11-21
2,1977-05-10,NICK,HEIDFELD,2025-11-21,2025-11-21
3,1985-06-27,NICO,ROSBERG,2025-11-21,2025-11-21
4,1981-07-29,FERNANDO,ALONSO,2025-11-21,2025-11-21
5,1981-10-19,HEIKKI,KOVALAINEN,2025-11-21,2025-11-21
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21



## Day 3 - Using python Delta table API

In [0]:
from pyspark.sql.functions import current_timestamp
from delta.tables import *

deltaTablePeople = DeltaTable.forPath(spark, "/mnt/formula1dlsaga/demo/drivers_merge")

# deltaTablePeopleUpdates = DeltaTable.forPath(spark, '/tmp/delta/people-10m-updates')
# dfUpdates = deltaTablePeopleUpdates.toDF()

deltaTablePeople.alias('tgt') \
  .merge(
    drivers_d3_df.alias('upd'),
    'tgt.driverId = upd.driverId'
  ) \
  .whenMatchedUpdate(set =
    {
      "dob": "upd.dob",
      "forename": "upd.forename",
      "surname": "upd.surname",
      "createdDate": "current_timestamp()"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "driverID": "upd.driverId",
      "dob": "upd.dob",
      "forename": "upd.forename",
      "surname": "upd.surname",
      "updatedDate": "current_timestamp()"
    }
  ) \
  .execute()

In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverId,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21
11,1977-01-28,TAKUMA,SATO,2025-11-21,2025-11-21
12,1985-07-25,NELSON,PIQUET JR.,2025-11-21,2025-11-21
13,1981-04-25,FELIPE,MASSA,2025-11-21,2025-11-21
14,1971-03-27,DAVID,COULTHARD,2025-11-21,2025-11-21
15,1974-07-13,JARNO,TRULLI,2025-11-21,2025-11-21



1. History and Versioning
2. Time Travel
3. Vaccum

In [0]:
%sql
DESC HISTORY f1_demo.drivers_merge;

In [0]:
%sql 
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 2;

driverId,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21
11,1977-01-28,TAKUMA,SATO,2025-11-21,null
12,1985-07-25,NELSON,PIQUET JR.,2025-11-21,null
13,1981-04-25,FELIPE,MASSA,2025-11-21,null
14,1971-03-27,DAVID,COULTHARD,2025-11-21,null
15,1974-07-13,JARNO,TRULLI,2025-11-21,null


In [0]:
%sql 
SELECT * FROM f1_demo.drivers_merge TIMESTAMP AS OF '2025-11-21T19:18:01.000+00:00';

driverId,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,LEWIS,HAMILTON,2025-11-21,2025-11-21
2,1977-05-10,NICK,HEIDFELD,2025-11-21,2025-11-21
3,1985-06-27,NICO,ROSBERG,2025-11-21,2025-11-21
4,1981-07-29,FERNANDO,ALONSO,2025-11-21,2025-11-21
5,1981-10-19,HEIKKI,KOVALAINEN,2025-11-21,2025-11-21
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21



pyspark style


In [0]:
df = spark.read.format("delta").option("timestampAsOf", "2025-11-21T19:15:04.000+00:00").load("/mnt/formula1dlsaga/demo/drivers_merge")

In [0]:
display(df)

driverId,dob,forename,surname,createdDate,updatedDate
1,1985-01-07,Lewis,Hamilton,2025-11-21,null
2,1977-05-10,Nick,Heidfeld,2025-11-21,null
3,1985-06-27,Nico,Rosberg,2025-11-21,null
4,1981-07-29,Fernando,Alonso,2025-11-21,null
5,1981-10-19,Heikki,Kovalainen,2025-11-21,null
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21



Delete data in version control ( GDPR ) rules. default 7 days. witch condition 0 hours. 

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;
VACUUM f1_demo.drivers_merge RETAIN 0 HOURS

path
dbfs:/mnt/formula1dlsaga/demo/drivers_merge



Restore data. Retrive from the latest vaccum.

In [0]:
%sql 
DELETE FROM f1_demo.drivers_merge WHERE driverId = 1;

num_affected_rows
1


In [0]:
%sql 
SELECT * FROM f1_demo.drivers_merge;

driverId,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21
11,1977-01-28,TAKUMA,SATO,2025-11-21,2025-11-21
12,1985-07-25,NELSON,PIQUET JR.,2025-11-21,2025-11-21
13,1981-04-25,FELIPE,MASSA,2025-11-21,2025-11-21
14,1971-03-27,DAVID,COULTHARD,2025-11-21,2025-11-21
15,1974-07-13,JARNO,TRULLI,2025-11-21,2025-11-21


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge VERSION AS OF 12;

driverId,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21
11,1977-01-28,TAKUMA,SATO,2025-11-21,2025-11-21
12,1985-07-25,NELSON,PIQUET JR.,2025-11-21,2025-11-21
13,1981-04-25,FELIPE,MASSA,2025-11-21,2025-11-21
14,1971-03-27,DAVID,COULTHARD,2025-11-21,2025-11-21
15,1974-07-13,JARNO,TRULLI,2025-11-21,2025-11-21


In [0]:
%sql
MERGE INTO f1_demo.drivers_merge AS tgt
USING f1_demo.drivers_merge VERSION AS of 12 AS src
  ON (tgt.driverId = src.driverId)
WHEN NOT MATCHED THEN
  INSERT *;

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1


In [0]:
%sql
SELECT * FROM f1_demo.drivers_merge;

driverId,dob,forename,surname,createdDate,updatedDate
6,1985-01-11,KAZUKI,NAKAJIMA,2025-11-21,2025-11-21
7,1979-02-28,SÉBASTIEN,BOURDAIS,2025-11-21,2025-11-21
8,1979-10-17,KIMI,RÄIKKÖNEN,2025-11-21,2025-11-21
9,1984-12-07,ROBERT,KUBICA,2025-11-21,2025-11-21
10,1982-03-18,TIMO,GLOCK,2025-11-21,2025-11-21
11,1977-01-28,TAKUMA,SATO,2025-11-21,2025-11-21
12,1985-07-25,NELSON,PIQUET JR.,2025-11-21,2025-11-21
13,1981-04-25,FELIPE,MASSA,2025-11-21,2025-11-21
14,1971-03-27,DAVID,COULTHARD,2025-11-21,2025-11-21
15,1974-07-13,JARNO,TRULLI,2025-11-21,2025-11-21



Transaction logs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_txn (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING DELTA

In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 1;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
INSERT INTO f1_demo.drivers_txn
SELECT * FROM f1_demo.drivers_merge
WHERE driverId = 2;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
DELETE FROM f1_demo.drivers_txn WHERE driverId = 1;

num_affected_rows
1


In [0]:
for driver_id in range(3, 20):
    spark.sql(f"""INSERT INTO f1_demo.drivers_txn
                  SELECT * FROM f1_demo.drivers_merge
                  WHERE driverId = {driver_id}""")


Convert Parquet to Delta

In [0]:
%sql
CREATE TABLE IF NOT EXISTS f1_demo.drivers_convert_to_delta (
  driverId INT,
  dob DATE,
  forename STRING,
  surname STRING,
  createdDate DATE,
  updatedDate DATE
)
USING PARQUET

In [0]:
%sql
INSERT INTO f1_demo.drivers_convert_to_delta
SELECT * FROM f1_demo.drivers_merge;


In [0]:
%sql
CONVERT TO DELTA f1_demo.drivers_convert_to_delta;

In [0]:
convert_df = spark.table("f1_demo.drivers_convert_to_delta")

In [0]:
convert_df.write.format("parquet").mode("overwrite").save("/mnt/formula1dlsaga/demo/drivers_convert_to_delta_new")

In [0]:
%sql
CONVERT TO DELTA parquet.`/mnt/formula1dlsaga/demo/drivers_convert_to_delta_new`